In [ ]:
! pip install streamlit==0.84.2
! pip install streamlit_option_menu

In [4]:
%%writefile dashbord.py
import math
import scipy
from scipy.optimize import fsolve
from scipy.interpolate import interp2d
from sympy import * 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import streamlit as st
import random
import plotly.express as px
from streamlit_option_menu import option_menu

h = 8.4  # height of building
thickness_slab = 175
width_beam = 300
span_depth_ratio = 10
depth_beam = 750
width_column_x = 450
width_column_y = 700
density = 25
wall_thickness_outer = 230
wall_thickness_inner = 115
wall_length_inner = 151.365
wall_length_outer = 183.036
wall_thickness = [wall_thickness_inner/1000,wall_thickness_outer/1000]
wall_length = [wall_length_inner,wall_length_outer]
density_wall = 20
length_plan = 31475
width_plan = 28470
live_load = 5
floor_finish = 2
percentage_opening = 25
cutout_area = 0
number_Column = 52
height_column_ground = 3.45
height_column_First = 4.20
height_column_Terrace = 1.95
length_Beam = 439.66
area_slab = 928.02
Z = 0.16
I = 1.5
T = 0.075*(h**0.75)
soil_type = "Medium"
Sa_g =  1/T   # Sa_g = Sa_g
R = 5
weight_column_G = density*number_Column*width_column_x/1000*width_column_y/1000*height_column_ground
weight_column_F = density*number_Column*width_column_x/1000*width_column_y/1000*height_column_First
weight_column_T = density*number_Column*width_column_x/1000*width_column_y/1000*height_column_Terrace
weight_slab = density*area_slab*thickness_slab/1000
net_weight_wall_inner_G = 0.75*density_wall*wall_thickness[0]*1.95*wall_length[0]
net_weight_wall_outer_G = 0.75*density_wall*wall_thickness[1]*1.95*wall_length[1]
net_weight_wall_inner_F = 0.75*density_wall*wall_thickness[0]*3.6*wall_length[0]
net_weight_wall_outer_F = 0.75*density_wall*wall_thickness[1]*3.6*wall_length[1]
net_weight_wall_inner_T = 0.75*density_wall*wall_thickness[0]*1.65*wall_length[0]
net_weight_wall_outer_T = 0.75*density_wall*wall_thickness[1]*1.65*wall_length[1]
net_weight_wall_parapet = density_wall*wall_thickness[1]*0.9*wall_length[1]
weight_beam_T = density*width_beam/1000*depth_beam/1000*length_Beam
weight_beam_F = density*width_beam/1000*depth_beam/1000*length_Beam
floor_finish_load = floor_finish*area_slab
imposed_load = 0.5*live_load*area_slab
total_seismic_weight_G = imposed_load + floor_finish_load + weight_column_G + weight_slab + net_weight_wall_inner_G + net_weight_wall_outer_G
total_seismic_weight_F = imposed_load + floor_finish_load + weight_slab + weight_column_F + net_weight_wall_inner_F + net_weight_wall_outer_F + weight_beam_F
total_seismic_weight_T = imposed_load + floor_finish_load + weight_column_T + weight_slab + net_weight_wall_inner_T + net_weight_wall_outer_T + net_weight_wall_parapet + weight_beam_T
total_seismic_weight = total_seismic_weight_G + total_seismic_weight_F + total_seismic_weight_T
Ah = (Z/2) *  (I/R) * (Sa_g)
base_shear = Ah * total_seismic_weight
h_G = 0
h_F = 4.41
h_T = 8.31

lateral_force_ground = ((total_seismic_weight_G * h_G**2)/(total_seismic_weight_G * h_G**2 + total_seismic_weight_F * h_F**2 + total_seismic_weight_T * h_T**2))*base_shear
lateral_force_first = ((total_seismic_weight_F * h_F**2)/(total_seismic_weight_G * h_G**2 + total_seismic_weight_F * h_F**2 + total_seismic_weight_T * h_T**2))*base_shear
lateral_force_terrace = ((total_seismic_weight_T * h_T**2)/(total_seismic_weight_G * h_G**2 + total_seismic_weight_F * h_F**2 + total_seismic_weight_T * h_T**2))*base_shear



with st.sidebar:
  selected = option_menu(
    menu_title = None,
    options = ["HOME","INPUT", "OUTPUT", "CALCULATION", "SOURCE"],
    icons = ["house-fill","display", "display" ,"calculator-fill", "server"],
  )

if selected == "HOME" :
  st.header('Welcome')
  st.write('-----')
  st.subheader('Objective')
  st.write("Estimating the seismic forces acting in the X- and Y- directions for the building by the equivalent static method,")

if selected == "INPUT" :
  st.write(f"{selected}")
  st.write("---")
  
  st.header('Building Characteristic')
  st.write("Height of building = ",h,"m")
  st.write("Thickness of Slab = ",thickness_slab,"mm")
  st.write("Width Of Beam = ",width_beam,"mm")  
  st.write("Depth of Beam = ",depth_beam,"mm") 
  st.write("Column Dimension = ",width_column_x,"x",width_column_y,"mm") 
  st.write("Wall Thickness at Outer Perimeter = ",wall_thickness_outer,"mm") 
  st.write("Wall Thickness at Inner Partition = ",wall_thickness_inner,"mm") 
  st.write("Density",density,"KN/m3") 
  st.write("Density of wall",density_wall,"KN/m3") 

  st.header('Loading Characteristic')
  st.write("Live Load",live_load,"KN/m2") 
  st.write("Floor Finish",floor_finish,"KN/m2") 

  st.header('Design Acceleration Spectrum')
  st.write("Seismic Zone Factor(Z)",Z)
  st.write("Importance Factor (I)",I)
  st.write("Natural Period",T) 
  st.write("Sa/g",Sa_g)
  st.write("Response Reduction Factor (R) = ",R)
  
      


if selected == "OUTPUT" :
  st.write(f"{selected}")
  st.write("-----")

  option = st.selectbox("Select",("Seismic Weight - Ground Floor","Seismic Weight - First Floor","Seismic Weight - Terrace","Design Horizontal Seismic Coefficient","Lateal Force"))
  if option == 'Seismic Weight - Ground Floor' :
    st.write("COLUMN")
    Column_G = {
      "Number" : [number_Column],
      "Dimension1" : [width_column_x],
      "Dimension2" : [width_column_y ],
      "length" : [height_column_ground],
      "Volume" : [number_Column*width_column_x/1000*width_column_y/1000*height_column_ground],
      "Weight" : [weight_column_G],
    }
    df = pd.DataFrame(Column_G)
    st.dataframe(df)

    st.write("WALL")
    Walls_G = {
      "Thickness" : [wall_thickness[0],wall_thickness[1]],
      "Height" : [1.95,1.95],
      "length" : [wall_length[0],wall_length[1]],
      "Volume" : [wall_thickness[0]*1.95*wall_length[0],wall_thickness[1]*1.95*wall_length[1]],
      "Weight" : [density_wall*wall_thickness[0]*1.95*wall_length[0],density_wall*wall_thickness[1]*1.95*wall_length[1]],
      "Net Weight" : [net_weight_wall_inner_G,net_weight_wall_outer_G]
    }
    df = pd.DataFrame(Walls_G)
    st.dataframe(df)

    st.write("SLAB")
    Slab_G = {
      "Thickness" : [thickness_slab],
      "Area" : [area_slab],
      "Volume" : [area_slab*thickness_slab/1000],
      "Weight" : [weight_slab]
    }
    df = pd.DataFrame(Slab_G)
    st.dataframe(df)

    st.write("Floor Finish Load = ",floor_finish_load,"KN")
    st.write("Imposed Load = ",imposed_load,"KN")
    st.write("Total Seismic Weight at Ground Floor = ", total_seismic_weight_G,"KN")

  if option == 'Seismic Weight - First Floor' :
    st.write("COLUMN")
    Column_F = {
      "Number" : [number_Column],
      "Dimension1" : [width_column_x],
      "Dimension2" : [width_column_y ],
      "length" : [height_column_First],
      "Volume" : [number_Column*width_column_x/1000*width_column_y/1000*height_column_First],
      "Weight" : [weight_column_F ],
    }
    df = pd.DataFrame(Column_F)
    st.dataframe(df)

    st.write("BEAM")
    Beam_F = {
      "Width" : [width_beam],
      "Depth" : [depth_beam],
      "length" : [length_Beam],
      "Volume" : [width_beam/1000*depth_beam/1000*length_Beam],
      "Weight" : [weight_beam_F],
    }
    df = pd.DataFrame(Beam_F)
    st.dataframe(df)  

    st.write("WALLS")
    Walls_F = {
      "Thickness" : [wall_thickness[0],wall_thickness[1]],
      "Height" : [3.6,3.6],
      "length" : [wall_length[0],wall_length[1]],
      "Volume" : [wall_thickness[0]*3.6*wall_length[0],wall_thickness[1]*3.6*wall_length[1]],
      "Weight" : [density_wall*wall_thickness[0]*3.6*wall_length[0],density_wall*wall_thickness[1]*3.6*wall_length[1]],
      "Net Weight" : [net_weight_wall_inner_F,net_weight_wall_outer_F]
    }
    df = pd.DataFrame(Walls_F)
    st.dataframe(df)  

    st.write("SLAB")
    Slab_F = {
      "Thickness" : [thickness_slab],
      "Area" : [area_slab],
      "Volume" : [area_slab*thickness_slab/1000],
      "Weight" : [weight_slab]
    }
    df = pd.DataFrame(Slab_F)
    st.dataframe(df)

    st.write("Floor Finish Load = ",floor_finish_load,"KN")
    st.write("Imposed Load = ",imposed_load,"KN")
    st.write("Total Seismic Weight at First Floor = ", total_seismic_weight_F,"KN")




  if option == 'Seismic Weight - Terrace' :
    st.write("COLUMN")
    Column_T = {
      "Number" : [number_Column],
      "Dimension1" : [width_column_x],
      "Dimension2" : [width_column_y ],
      "length" : [height_column_Terrace],
      "Volume" : [number_Column*width_column_x/1000*width_column_y/1000*height_column_Terrace],
      "Weight" : [weight_column_T ],
    }
    df = pd.DataFrame(Column_T)
    st.dataframe(df)

    st.write("BEAM")
    Beam_T = {
      "Width" : [width_beam],
      "Depth" : [depth_beam],
      "length" : [length_Beam],
      "Volume" : [width_beam/1000*depth_beam/1000*length_Beam],
      "Weight" : [weight_beam_T],
    }
    df = pd.DataFrame(Beam_T)
    st.dataframe(df)   
    
    st.write("WALL")
    Walls_T = {
      "Thickness" : [wall_thickness[0],wall_thickness[1],wall_thickness[1]],
      "Height" : [1.65,1.65,0.9],
      "length" : [wall_length[0],wall_length[1],wall_length[1]],
      "Volume" : [wall_thickness[0]*1.65*wall_length[0],wall_thickness[1]*1.65*wall_length[1],wall_thickness[1]*0.9*wall_length[1]],
      "Weight" : [density_wall*wall_thickness[0]*1.65*wall_length[0],density_wall*wall_thickness[1]*1.65*wall_length[1],density_wall*wall_thickness[1]*0.9*wall_length[1]],
      "Net Weight" : [net_weight_wall_inner_T ,net_weight_wall_outer_T,net_weight_wall_parapet]
    }
    df = pd.DataFrame(Walls_T)
    st.dataframe(df)
    
    st.write("SLAB")
    Slab_T = {
      "Thickness" : [thickness_slab],
      "Area" : [area_slab],
      "Volume" : [area_slab*thickness_slab/1000],
      "Weight" : [weight_slab]
    }
    df = pd.DataFrame(Slab_T)
    st.dataframe(df)

    st.write("Floor Finish Load = ",floor_finish*area_slab,"KN")
    st.write("Imposed Load = ",0.5*live_load*area_slab,"KN")
    st.write("Total Seismic Weight at Terrace = ", total_seismic_weight_T,"KN")

  if option == 'Design Horizontal Seismic Coefficient' :
    st.write("Seismic Zone Factor(Z)",Z)
    st.write("Importance Factor (I)",I)
    st.write("Natural Period",T) 
    st.write("Sa/g",Sa_g)
    st.write("Response Reduction Factor (R) = ",R)
    Ah = (Z/2) *  (I/R) * (Sa_g)
    st.write("Design Horizontal Seismic Coefficient",Ah)   
  
  if option == 'Lateal Force' :
    st.write("Ground Floor Seismic Weight = ",total_seismic_weight_G,"KN")
    st.write("First Floor Seismic Weight = ",total_seismic_weight_F,"KN")
    st.write("Terrace Floor Seismic Weight = ",total_seismic_weight_T,"KN")
    st.write("Total Sesismic Weight = ",total_seismic_weight,"KN")
    st.write("Base Shear = ",base_shear,"KN")
    st.write("Lateral Force at Ground Floor = ",lateral_force_ground,"KN")
    st.write("Lateral Force at First Floor = ",lateral_force_first,"KN")
    st.write("Lateral Force at Terrace  = ",lateral_force_terrace,"KN")

if selected == "CALCULATION" :
  st.write(f"{selected}")
  st.write("-----")  
  st.header("Design Horizontal Seismic Coefficient")
  st.write("Seismic Zone Factor (Z) = ",Z," IS 1893:2016 -> Clause 6.4.2 -> Table 3")
  st.write("Importance Factor (I) = ",I," IS 1893:2016 -> Clause 7.2.3 -> Table 8")
  st.write("Natural Period (T) = ",T," IS 1893:2016 -> Clause 7.6.2.a ")
  st.write("Soil type = ","Rocky or hard soil")  
  st.write("Sa/g = ",Sa_g," IS 1893:2016 -> Clause 6.4.2.a")
  st.write("Sa/g = ",Sa_g," IS 1893:2016 -> Clause 6.4.2.a")  
  st.write("Response Reduction Factor (R)",R," IS 1893:2016 -> Clause 7.2.6 -> Table 9")
  st.write("Design Horizontal Seismic Coefficient",Ah," IS 1893:2016 -> Clause 6.4.2")
  st.header("Lateral Forces")
  st.write("Ground Floor Seismic Weight = ",total_seismic_weight_G,"KN")
  st.write("First Floor Seismic Weight = ",total_seismic_weight_F,"KN")
  st.write("Terrace Floor Seismic Weight = ",total_seismic_weight_T,"KN")
  st.write("Total Sesismic Weight = ",total_seismic_weight,"KN")
  st.write("Base Shear = ",base_shear,"KN")
  st.write("Lateral Force at Ground Floor = ",lateral_force_ground,"KN")
  st.write("Lateral Force at First Floor = ",lateral_force_first,"KN")
  st.write("Lateral Force at Terrace  = ",lateral_force_terrace,"KN")
  st.write("Lateral Forces will be same in both x and y direction.") 

if selected == "SOURCE" :
  st.write(f"{selected}")
  st.write("---")
  st.write("Is Code 1893 Part 1-2016")
  st.write("[Learn More >](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwjpnuPk1vr6AhX_U2wGHR9WDjQQFnoECA4QAQ&url=https%3A%2F%2Fnitsri.ac.in%2FDepartment%2FCivil%2520Engineering%2FCSE_202_Chapter_4-1893-part-1-2016-1pdf.pdf&usg=AOvVaw0Vixp62pexHBRU6Y_RKmcg)")


Overwriting dashbord.py


In [ ]:
! streamlit run dashbord.py & npx localtunnel --port 8501